In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
# timeのインポート
import time

In [3]:
np.random.seed(42)

In [4]:
n_rows, n_cols = 40, 100
n = 1000
C = np.random.uniform(10, 50, size=(n_rows, n_cols))
T = np.ones((1, n_rows))
xi = []
for _ in range(n):
    mean = np.random.uniform(0, 10, n_cols)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, n_cols)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), n_cols))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
xi = np.array(xi)
M = np.array([100] * n_rows)

In [5]:
lambda_ = 1.0
alpha = 0.01
max_iter = 1000
tolerance = 1e-4
K = 5

In [12]:
# L1ノルムの計算
def l1_norm(z):
    return np.sum(np.abs(z))

# 最大Kノルムの計算
def lK_norm(z, K):
    return np.sum(np.sort(np.abs(z))[:K])
# np.sum(np.sort(np.abs(z))[:K])で小さい方からK番目までの和を計算

# ソフト閾値作用素の計算
def soft_thresholding(z, lambda_):
        if z <= -lambda_:
            return z + lambda_
        elif z >= lambda_:
            return z - lambda_
        else:
            return 0

# プロキシ関数の計算
# zの最大K要素のインデックス集合に含まれていればzを，含まれていなければソフト閾値作用素を適用したzを返す
def prox_operator(z, lambda_, K):
    sorted_indices = np.argsort(np.abs(z))[::-1]
    top_K_indices = sorted_indices[:K]
    z_prox = z.copy()
    for i in range(len(z)):
        if i not in top_K_indices:
            z_prox[i] = soft_thresholding(z[i], lambda_)
    return z_prox

def objective_function(C, x, z, lambda_):
    return np.sum(C.T @ x) + lambda_ * lK_norm(z, K)

In [17]:
z = [1, 2, 3, 4, 5, 4, 5]
print(z)

[1, 2, 3, 4, 5, 4, 5]


In [18]:
l1_norm(z)

24

In [19]:
lK_norm(z, 2)

3

In [25]:
def proximal_gradient_method(C, T, xi, lambda_, alpha, max_iter, tolerance, K):
    n_rows, n_cols = C.shape
    x = np.random.rand(n_rows, n_cols)
    z = np.random.rand(xi.shape[0])

    for k in range(max_iter):
        # 勾配計算
        grad_x = C
        # grad_z = lambda_ * np.sign(z)

        obj = objective_function(C, x, z, lambda_)

        # 勾配降下ステップ
        x_new = x - alpha * grad_x

        # xに非負制約を課す
        x_new = np.maximum(x_new, 0)

        # 制約を満たすためのzの更新
        v = T @ x_new
        for i in range(n):
            for j in range(n_cols):
                if  v[0, j]< (1-z[i]) * xi[i, j]:
                    z[i] = 1
                else:
                    z[i] = 0

        # 供給制約を課す
        for k in range(n_rows):
            if np.sum(x_new[:, k]) > M[k]:
                x_new[:, k] = x_new[:, k]  * (M[k] / np.sum(x_new[:, k]))

        # 近接演算子の適用
        z_new = prox_operator(z, lambda_, K)

        obj_new = objective_function(C, x_new, z_new, lambda_)

        # 収束判定
        if np.abs(obj - obj_new) < tolerance:
            print(f"Converged at iteration {k}")
            break
        x = x_new
        z = z_new
        obj = obj_new


    return x, z, obj

In [26]:
def objective_function(C, x, z, lambda_):
    return np.sum(C.T @ x) + lambda_ * lK_norm(z, K)

x = np.random.rand(n_rows, n_cols)
z = np.random.rand(xi.shape[0])
obj = objective_function(C, x, z, lambda_)
print(obj)

5967389.625832581


In [27]:
x_estimated, z_estimated, _ = proximal_gradient_method(C, T, xi, lambda_, alpha, max_iter, tolerance, K)
print("Estimated X:", x_estimated)
print("Estimated Z:", z_estimated)
print(np.sum(C.T @ x_estimated))

Converged at iteration 39
Estimated X: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Estimated Z: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [107]:
print(np.sum(C.T @ x_estimated))

0.0


In [28]:
# z_estimatedに含まれる1の数をカウント
print(np.sum(z_estimated))

5.0


In [23]:
x = np.random.rand(n_rows, n_cols)
print(x)

[[0.16438457 0.82695214 0.47144826 ... 0.78615866 0.96610922 0.53876057]
 [0.69787439 0.25172611 0.3331356  ... 0.85562444 0.40297005 0.83289373]
 [0.88602484 0.40679922 0.78842633 ... 0.87254545 0.81402135 0.93397844]
 ...
 [0.05254419 0.04383909 0.39326561 ... 0.58682135 0.53431615 0.33578677]
 [0.78042674 0.4835413  0.40440436 ... 0.05099072 0.57173778 0.47067494]
 [0.66036178 0.30840253 0.1507861  ... 0.14778021 0.07645942 0.52718289]]


In [24]:
x[:, 0]

array([0.16438457, 0.69787439, 0.88602484, 0.27407522, 0.37707189,
       0.1372434 , 0.25263737, 0.97318157, 0.94714761, 0.93882302,
       0.73481466, 0.90177116, 0.97685585, 0.87522301, 0.67548872,
       0.19674396, 0.27886957, 0.95175346, 0.15390919, 0.02787795,
       0.03646922, 0.94552003, 0.45675351, 0.92582551, 0.05623839,
       0.67546559, 0.96577973, 0.761734  , 0.3370085 , 0.56689591,
       0.48373795, 0.40064515, 0.2178461 , 0.70928579, 0.27462774,
       0.2048223 , 0.78549466, 0.05254419, 0.78042674, 0.66036178])

In [34]:
np.dot(T, x[:, 0])

array([21.7192542])

In [37]:
v = T @ x
print(v)

[[21.7192542  20.60511549 19.92415038 21.96813086 19.33181797 20.96391537
  19.02401427 18.77618362 20.08480686 20.97100437 20.75222335 19.19700366
  21.14160784 22.97733758 19.73338445 19.67707031 21.21587174 16.79745017
  16.33180617 20.73149574 19.61016219 18.03349607 20.65320792 18.95364257
  16.33663593 18.88343262 18.30954911 16.86752428 19.84404412 19.71665946
  20.83393144 20.97865728 19.18162326 21.58610459 20.91661877 17.71759242
  18.88271056 20.79051544 20.11759247 18.24035686 18.54089064 22.29697663
  17.2847903  19.33899776 22.15665851 21.76635775 19.46815911 17.87642747
  21.69356759 18.22932009 21.33679318 20.86237091 17.90443861 21.4093848
  19.58028375 20.36714403 22.04596184 20.07847842 16.07103291 23.5229644
  20.80991376 18.14417407 19.38067616 21.62594263 21.18043718 17.45732854
  21.52219993 21.44289852 19.06256277 20.29981537 19.15939346 18.17154839
  18.4868753  20.38386767 18.89393388 19.96037344 20.12155865 18.24522139
  20.2352711  21.05177505 21.31086617 20

In [39]:
v[0, 0]

21.719254196515955

In [26]:
xi[0]

array([5.04126479e+00, 8.73932240e+00, 7.50529322e+00, 2.58763600e-01,
       3.15031371e+00, 7.11739088e+00, 4.04318092e+00, 3.09351094e+00,
       3.48874012e+00, 1.03597496e+00, 2.33850793e+00, 4.74227312e+00,
       6.96891986e+00, 1.24727496e+00, 4.40011449e+00, 3.88169201e+00,
       1.08353421e+01, 6.45188127e+00, 4.96691666e+00, 3.42764421e+00,
       6.17639242e+00, 4.15392040e-02, 4.06439567e+00, 7.94110658e+00,
       7.75748409e+00, 1.17014147e+01, 5.33148871e+00, 1.03238528e+00,
       5.86294820e+00, 4.83747834e+00, 5.31877793e+00, 4.93426863e+00,
       2.69178373e-01, 1.29163011e+01, 2.80907816e+00, 9.74283393e+00,
       8.74508829e+00, 7.42182976e+00, 7.41367277e+00, 5.91665856e+00,
       7.74667628e+00, 2.68896518e+00, 4.76904146e+00, 1.76629994e+00,
       5.10239494e+00, 2.41619312e+00, 1.93152695e+00, 2.91731771e+00,
       4.95594214e+00, 7.35734935e-01, 8.48384809e+00, 8.84425568e+00,
       7.20107626e+00, 5.83878572e+00, 8.36406564e+00, 2.96407464e-01,
      

In [27]:
T @ x

array([[21.7192542 , 20.60511549, 19.92415038, 21.96813086, 19.33181797,
        20.96391537, 19.02401427, 18.77618362, 20.08480686, 20.97100437,
        20.75222335, 19.19700366, 21.14160784, 22.97733758, 19.73338445,
        19.67707031, 21.21587174, 16.79745017, 16.33180617, 20.73149574,
        19.61016219, 18.03349607, 20.65320792, 18.95364257, 16.33663593,
        18.88343262, 18.30954911, 16.86752428, 19.84404412, 19.71665946,
        20.83393144, 20.97865728, 19.18162326, 21.58610459, 20.91661877,
        17.71759242, 18.88271056, 20.79051544, 20.11759247, 18.24035686,
        18.54089064, 22.29697663, 17.2847903 , 19.33899776, 22.15665851,
        21.76635775, 19.46815911, 17.87642747, 21.69356759, 18.22932009,
        21.33679318, 20.86237091, 17.90443861, 21.4093848 , 19.58028375,
        20.36714403, 22.04596184, 20.07847842, 16.07103291, 23.5229644 ,
        20.80991376, 18.14417407, 19.38067616, 21.62594263, 21.18043718,
        17.45732854, 21.52219993, 21.44289852, 19.0

In [46]:
z = np.random.randint(0, 2, size=n_rows)
print(z)
sorted_indices = np.argsort(np.abs(z))
print(sorted_indices)

[1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 1
 0 1 1]
[31 17 16 15 14 13 24 10 21 33 35  6 23  4  3 37  1 34 29 27  0 30 32 36
 28 26 19 22 20 38 18 12 11  9  8  7  5  2 25 39]


In [42]:
v.shape

(1, 100)

In [43]:
xi.shape

(1000, 100)

In [51]:
z_rand = np.random.randn(10)
print(z_rand)
sorted_indices = np.argsort(z_rand)[::-1]
print(sorted_indices)

[ 2.11766401 -0.82855483 -0.73850172 -0.08412071  0.03978997  0.06324368
  0.22570606  1.27352913  0.68288789  0.71534999]
[0 7 9 8 6 5 4 3 2 1]


In [55]:
for i in sorted_indices:
    print(i)
    print(z_rand[i])

0
2.1176640124207458
7
1.2735291309559527
9
0.7153499942215903
8
0.6828878870956991
6
0.22570606324878723
5
0.06324368197117668
4
0.03978996986452733
3
-0.08412071008103335
2
-0.7385017211954148
1
-0.8285548302626455


In [54]:
z_rand[sorted_indices]

array([ 2.11766401,  1.27352913,  0.71534999,  0.68288789,  0.22570606,
        0.06324368,  0.03978997, -0.08412071, -0.73850172, -0.82855483])

In [60]:
print(z_rand)

[ 2.11766401 -0.82855483 -0.73850172 -0.08412071  0.03978997  0.06324368
  0.22570606  1.27352913  0.68288789  0.71534999]


In [62]:
soft_thresholding(z_rand,0.1)

array([ 2.01766401, -0.72855483, -0.63850172,  0.        ,  0.        ,
        0.        ,  0.12570606,  1.17352913,  0.58288789,  0.61534999])

In [91]:
z_rand = np.random.randn(10)
print(z_rand)
sorted_indices = np.argsort(z_rand)[::-1]
print(sorted_indices)
prox_operator(z_rand, 0.1, 2)

[ 0.79621658 -0.61317463 -0.34479138  0.96643159 -0.63967167  0.41734225
  0.25811987 -0.37749277  0.50772102 -0.82321583]
[3 0 8 5 6 2 7 1 4 9]


array([ 0.79621658, -0.51317463, -0.24479138,  0.96643159, -0.53967167,
        0.31734225,  0.15811987, -0.27749277,  0.40772102, -0.82321583])

In [70]:
sorted_indices[0] <= 0

False

In [94]:
z_int = [1, 2, 4, 3, 5]
s = np.argsort(z_int)[::-1]
print(s)
prox_operator(z_int, 0.1, 2)

[4 2 3 1 0]


[0.9, 1.9, 4, 2.9, 5]

In [87]:
for i in range(len(z_int)):
    print(z_int[i] + 0.1)

1.1
2.1
4.1
3.1
5.1


DCAの劣勾配計算

In [35]:
def calculate_subgradient(w, K):
    n = len(w)
    s = np.zeros(n)
    w = np.sort(np.abs(w))[::-1]
    s[:K] = np.sign(w[:K])
    return w, s

In [36]:
np.random.seed(42)
w = np.random.randn(10)
print(w)

[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
  1.57921282  0.76743473 -0.46947439  0.54256004]


In [37]:
calculate_subgradient(w, 3)

(array([1.57921282, 1.52302986, 0.76743473, 0.64768854, 0.54256004,
        0.49671415, 0.46947439, 0.23415337, 0.23413696, 0.1382643 ]),
 array([1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]))

In [38]:
type(np.random.rand(10))

numpy.ndarray

In [39]:
type(np.array([1, 2, 3]))

numpy.ndarray